In [67]:
%reload_ext autoreload
%autoreload 2


In [68]:
#1. loads necessary packages

import numpy as np
from scipy import stats
import pandas as pd
import math
import os
from datetime import date

import sys
import csv

from Classes import DataO, MapO, GC




In [69]:
#2. set paths for raw data retreaval and result storing

today = date.today()

# dd/mm/YY
d1 = today.strftime("%d%m%Y")

resultpath="../6_Results/"+d1+"_Maps/"

#writing impact results
rows=[]

In [70]:
#3. Gen thresholds and correlation tables

#### THRESHOLDS #####
from numpy import NaN


thresh_df=pd.DataFrame(
    columns=["Maxtemp",
            "Avgtemp"],
    index= ["Gas",
            "Nuclear",
            "Solar PV",
            "Transformers",
            "Overhead Lines",
            "Demand"]
)

thresh_df['Maxtemp']=[16,20,25,20,23.08,NaN]
thresh_df['Avgtemp']=[NaN,NaN,NaN,NaN,NaN,15]

'TESTING'
print(thresh_df)

#### CORRELATIONS #####
correl_df = pd.DataFrame(
    columns=["Maxtemp",
            "Avgtemp"],
    index= ["Gas",
            "Nuclear",
            "Solar PV",
            "Transformers",
            "Overhead Lines",
            "Demand"]
)

correl_df.loc["Gas","Maxtemp"]= lambda x : 0.7587*x-11.37
correl_df.loc["Nuclear","Maxtemp"]= lambda x : 1.03*(x-20)
correl_df.loc["Solar PV","Maxtemp"]= lambda x : 0.48*(x-25)
correl_df.loc["Transformers","Maxtemp"]= lambda x : 1*(x-20)
correl_df.loc["Overhead Lines","Maxtemp"]= lambda x : 1.3*x-30
correl_df.loc["Demand","Avgtemp"]= lambda x : 3.7*(x-15)

print(correl_df)


                Maxtemp  Avgtemp
Gas               16.00      NaN
Nuclear           20.00      NaN
Solar PV          25.00      NaN
Transformers      20.00      NaN
Overhead Lines    23.08      NaN
Demand              NaN     15.0
                                              Maxtemp  \
Gas             <function <lambda> at 0x7fe90d08ec10>   
Nuclear         <function <lambda> at 0x7fe90d08e4c0>   
Solar PV        <function <lambda> at 0x7fe90d08eee0>   
Transformers    <function <lambda> at 0x7fe90d08e550>   
Overhead Lines  <function <lambda> at 0x7fe90d08e670>   
Demand                                            NaN   

                                              Avgtemp  
Gas                                               NaN  
Nuclear                                           NaN  
Solar PV                                          NaN  
Transformers                                      NaN  
Overhead Lines                                    NaN  
Demand          <function <lambda

In [71]:
############################## PERIOD 2080-2100 #################################

#4. Initialises Data objects

#setting resolutions

spatial_res=60
time_res='daily'
start_year=2080
finish_year=2099 #finish year is included

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="60_Daily_Maxtemp_2080_2099_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Maxtemp_2080_2099/All",
                        respath=resultpath)

Avgtemp=DataO.DataObject(title="60_Daily_Avgtemp_2080_2099_UK",
                        varID="Avgtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Avgtemp_2080_2099/All",
                        respath=resultpath)

datao_list=[Maxtemp, Avgtemp]

'TESTING'
for object in datao_list:
    assert object.untreated_array.shape == (12,7170,1,75), 'Loaded data array does not have expected shape'


#5. Statistics test and calculations

Maxtemp.run_stats(KStest=False,stat_source='p90',tp_90=True,tp_10=False)
Avgtemp.run_stats(KStest=False,stat_source='p90',tp_90=True,tp_10=False)

max_temps=[Maxtemp.member_max, Avgtemp.member_max] #collecting each location's max temperature across time

#6. Initlises Map objects and builds max 20y value maps for Maxtemp and Avgtemp
Map_60_Daily_2080_2099=MapO.MapObject_daily(MapID="Maxtemp and Avtemp - max value in 20y",respath=resultpath,coord_dict=Maxtemp.coord_dict)


time_max=np.amax(max_temps[0],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2080_2099.geojson_props('maxtemp - 20y max',ftime_max)

time_max=np.amax(max_temps[1],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2080_2099.geojson_props('avgtemp - 20y max',ftime_max)

#7. Initialises Grid Component Objects

Gas=GC.GComponent("Gas")
Nuclear=GC.GComponent("Nuclear")
SolarPV=GC.GComponent("Solar PV")
Transf=GC.GComponent("Transformers")
OLines=GC.GComponent("Overhead Lines")
Demand=GC.GComponent("Demand")

component_list=[Gas,Nuclear,SolarPV,Transf,OLines,Demand]

datao_list=[Maxtemp, Avgtemp]

for gc in component_list:
    for wv in datao_list:

        gc.init_thresh(thresh_df)
        gc.init_correl(correl_df,datao_list)
        
        #8. Prompts impact array generation
        thresh=gc.thresholds.loc[wv.varID]
        gc.calc_impacts(thresh,wv)

    fmax_impact_array=gc.impact_arrays["max impact"].flatten('F')

    #9. writing impact statistics in a results file
    [min, max, avg]=gc.calc_impacts_abs()
    rows.append(["2080-2099", gc.GCID, min, max, avg])

    #10. geojson file generation and writing  
    Map_60_Daily_2080_2099.geojson_props('impact: -'+ gc.GCID + ' ' ,fmax_impact_array)

Map_60_Daily_2080_2099.geojson_write('2080_2099_impacts_max')


no correlation provided for: Gas- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Nuclear- variable Avgtemp impact calculation unsuccesful


KeyError: 'Hydroelectric'

In [ ]:
############################## PERIOD 2030-2050 #################################

#4. Initialises Data objects

#setting resolutions

spatial_res=60
time_res='daily'
start_year=2030
finish_year=2050 #finish year is included

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="60_Daily_Maxtemp_2030_2050_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7560,
                        fdpath="../3_Raw_Data/Maxtemp_2030_2050/All",
                        respath=resultpath)

Avgtemp=DataO.DataObject(title="60_Daily_Avgtemp_2030_2050_UK",
                        varID="Avgtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7560,
                        fdpath="../3_Raw_Data/Avgtemp_2030_2050/All",
                        respath=resultpath)

datao_list=[Maxtemp, Avgtemp]

'TESTING'
for object in datao_list:
    assert object.untreated_array.shape == (12,7560,1,75), 'Loaded data array does not have expected shape'


#5. Statistics test and calculations

Maxtemp.run_stats(KStest=False,stat_source='p90',tp_90=True,tp_10=False)
Avgtemp.run_stats(KStest=False,stat_source='p90',tp_90=True,tp_10=False)

max_temps=[Maxtemp.member_max, Avgtemp.member_max] #collecting each location's max temperature across time

#6. Initlises Map objects and builds max 20y value maps for Maxtemp and Avgtemp
Map_60_Daily_2030_2050=MapO.MapObject_daily(MapID="Maxtemp and Avtemp - max value in 20y",respath=resultpath,coord_dict=Maxtemp.coord_dict)


time_max=np.amax(max_temps[0],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2030_2050.geojson_props('maxtemp - 20y max',ftime_max)

time_max=np.amax(max_temps[1],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2030_2050.geojson_props('avgtemp - 20y max',ftime_max)


#7. Initialises Grid Component Objects

Gas=GC.GComponent("Gas")
Nuclear=GC.GComponent("Nuclear")
SolarPV=GC.GComponent("Solar PV")
Transf=GC.GComponent("Transformers")
OLines=GC.GComponent("Overhead Lines")
Demand=GC.GComponent("Demand")


component_list=[Gas,Nuclear,SolarPV,Transf,OLines,Demand]

datao_list=[Maxtemp, Avgtemp]

for gc in component_list:
    for wv in datao_list:

        gc.init_thresh(thresh_df)
        gc.init_correl(correl_df,datao_list)
        
        #8. Prompts impact array generation
        thresh=gc.thresholds.loc[wv.varID]
        gc.calc_impacts(thresh,wv)

    fmax_impact_array=gc.impact_arrays["max impact"].flatten('F')
        
    Map_60_Daily_2030_2050.geojson_props('impact: -'+ gc.GCID + ' ' ,fmax_impact_array)


    #9. writing impact statistics in a results file
    [min, max, avg]=gc.calc_impacts_abs()
    rows.append(["2030-2050", gc.GCID, min, max, avg])
    #10. geojson file generation and writing  
    Map_60_Daily_2030_2050.geojson_props('impact: -'+ gc.GCID + ' ' ,fmax_impact_array)

#10. geojson file generation and writing  
Map_60_Daily_2030_2050.geojson_write('2030_2050_impact_max')


no correlation provided for: Gas- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Nuclear- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Hydroelectric- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Solar PV- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Transformers- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Poles- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Overhead Lines- variable Avgtemp impact calculation unsuccesful
no correlation provided for: Demand- variable Maxtemp impact calculation unsuccesful


In [ ]:
# name of csv file 
from operator import concat


filename = concat(resultpath,"MinMaxAvg_Impacts.csv")

# field names 
fields = ['Period', "Grid Component" 'Min', 'Max', 'Avg'] 

# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)